In [1]:
import numpy as np
import scipy.stats as stats
import scipy.special
#graphing
import matplotlib.pyplot as plt

#optimize
from scipy.optimize import minimize

#interpolate
from scipy.integrate import quad

In [2]:
def optimal_reserve(mu,sigma):
    v = 0
    obj = lambda p :  -1*(p-v)*(1-stats.logistic(loc=mu,scale=sigma).cdf(p))
    result = minimize(obj,[0])
    return result.x

In [3]:
or1 = optimal_reserve(1.9450,2.0663)
or2 = optimal_reserve(6.6360,1.2836 )

print(or1, or2, (or1- or2)/2 )

[3.19481275] [5.20311742] [-1.00415233]


In [5]:
def expected_rev(p,mu = 3,sigma = 2,i=10):
   
    values_distr = stats.logistic(loc=mu, scale=sigma)
    cdf = lambda x : values_distr.cdf(x)
    pdf = lambda x: values_distr.pdf(x)
    factorial = scipy.special.factorial(i)/scipy.special.factorial(i-2)  
        
    order_cdf = lambda x : factorial*pdf(x)*cdf(x)**(i-2)*(1-cdf(x)) #np.log(i) + np.log(cdf) 
    integrand = lambda x : x*order_cdf(x)
    I = quad(integrand, p,100)
    return I


er1 = expected_rev(or1,i=7)
er2 = expected_rev(or2,i=7)
print(er1[0],er2[0], (er1[0]-er2[0])/er1[0] )

5.365106300361035 3.7786933537717795 0.2956908694395301


In [10]:
def marginal_effect(beta0,beta1,sigma,x):
    mux = beta0+beta1*x
    return stats.norm.cdf(mux/np.sqrt(np.abs(sigma)))*beta1



beta01 = 1.0968
beta11 = 0.5078

beta02 = 1.4116
beta12 = 0.4498

for x in [-3,0,3]:
    m1 = marginal_effect(beta01,beta11,1,x)
    m2 = 0.4498
    print(x,m1,m2,(m1-m2)/m1)

-3 0.17002939015417098 0.4498 -1.6454250032429818
0 0.4385541500100251 0.4498 -0.025643013501793974
3 0.5055687721981598 0.4498 0.1103089733087806


In [12]:
def avg_effect(beta0,beta1,sigma):
    integrand = lambda x: marginal_effect(beta0,beta1,sigma,x)*stats.norm(loc=5,scale=5).pdf(x)
    I = quad(integrand, -100,100)
    return I


beta01 = 1.0968
beta11 = 0.5078

beta02 = 1.4116
beta12 = 0.4498

a1 = avg_effect(beta01,beta11,1)
a2= 0.4498

print(a1[0],a2,(a1[0]-a2)/a1[0])

0.461402259012677 0.4498 0.02514564847927686


In [ ]:
support_x = np.arange(-15,20,.05)
me = marginal_effect(beta0,beta1,sigma, support_x )

me_95 = marginal_effect(beta0+ci[0]*1.96,beta1+ci[1]*1.96,
                        sigma+ci[2]*1.96, support_x )
me_05 = marginal_effect(beta0-ci[0]*1.96,beta1-ci[1]*1.96,
                        sigma-ci[2]*1.96, support_x )

#beta0 = 0.9793
#beta1 = 0.2559
#sigma = 1.0375
#ci = 0.047, 0.007, 0.049

beta0 = 1.0968
beta1 = 0.2431
sigma = 0.8248
#ci = 0.041, 0.006, 0.037

plt.plot(support_x, me ,label="mean",
         color='black')
plt.plot(support_x, me_95 ,label="95% confidence",
         color='black',linestyle='--')
plt.plot(support_x, me_05,
         color='black',linestyle='--')

plt.legend()

plt.ylabel("marginal effect")
plt.xlabel("xlabel")
plt.title("Tobit marginal effects")
plt.savefig('figs/me_weakover.png')